<a href="https://colab.research.google.com/github/cosmo3769/SSL-study/blob/classifier2/classifier_iNaturalist_aves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References

[Building a data pipeline](https://cs230.stanford.edu/blog/datapipeline/)

[tf.data](https://www.tensorflow.org/guide/data)

[How to use dataset in tensorflow](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428)

## Setting up kaggle service to fetch the dataset

Go to your kaggle account. Generate an API token. The file named "kaggle.json" will be downloaded to your local system. Upload the file **kaggle.json** in the colab so to use the kaggle service in colab.  

In [1]:
# Install the kaggle library.

%%capture
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

## Dataset

The dataset is taken from kaggle cometition on [Semi-Supervised Recognition Challenge - FGVC7](https://www.kaggle.com/competitions/semi-inat-2020/data). Here is the [GitHub page](https://github.com/cvl-umass/semi-inat-2020) giving the explanation of the dataset.

Some important points to note about dataset: 

| Split	| Details	| Classes	| Images |
| ----- | ------- | ------- | ------ |
| Train	| Labeled	| 200	    | 3,959  |
| Train	| Unlabeled, in-class	| 200	| 26,640 |
| Train	| Unlabeled, out-of-class |	-	| 122,208 |
| Val	  | Labeled	| 200	| 2,000 |
| Test | Public	| 200	| 4,000 |
| Test | Private | 200 | 4,000 |

In [5]:
! kaggle competitions download -c semi-inat-2020

100% 14.3G/14.3G [01:14<00:00, 248MB/s]
100% 14.3G/14.3G [01:14<00:00, 205MB/s]


In [6]:
%%capture
! unzip semi-inat-2020.zip

In [7]:
import os 

ANNOTATION_DIR = '/content/annotation/'
# os.listdir(ANNOTATION_DIR)

TRAINVAL_LABELLED_DIR = '/content/trainval_images/trainval_images/'
# os.listdir(TRAINVAL_LABELLED_DIR)

TEST_DIR = '/content/test/test/'
# os.listdir(TEST_DIR)

## Annotation Format

The dataset follows the annotation format of the COCO dataset. It is stored in the [JSON Format](https://www.json.org/json-en.html) and are organized as follows: 

```
{
  "info" : info,
  "images" : [image],
  "annotations" : [annotation],
}

info{
  "year" : int,
  "version" : str,
  "description" : str,
  "contributor" : str,
  "url" : str,
  "date_created" : datetime,
}

image{
  "id" : int,
  "width" : int,
  "height" : int,
  "file_name" : str
}

annotation{
  "id" : int,
  "image_id" : int,
  "category_id" : int
}

```



## Labelled training annotations

Showing the **annotations of labelled training images** from the annotation file [anno_l_train.json](/content/annotation/annotation/anno_l_train.json).

In [8]:
import json
import pandas as pd
from pandas import json_normalize

file = ANNOTATION_DIR + 'annotation/anno_l_train.json'

# load data using Python JSON module
with open(file,'r') as f:
    data = json.loads(f.read())
# Flatten data
annotations_labelled_training = pd.json_normalize(data, record_path =['annotations'])

annotations_labelled_training

,image_id,id,category_id
0,0,0,0
1,1,1,0
2,2,2,0
3,3,3,0
4,4,4,0
...,...,...,...
3954,3954,3954,199
3955,3955,3955,199
3956,3956,3956,199
3957,3957,3957,199


Showing the **images annotation of labelled training images** from the file [anno_l_train.json](/content/annotation/annotation/anno_l_train.json).

In [9]:
import json
import pandas as pd
from pandas import json_normalize

file = ANNOTATION_DIR + 'annotation/anno_l_train.json'

# load data using Python JSON module
with open(file,'r') as f:
    data = json.loads(f.read())
# Flatten data
images_annotations_labelled_training = pd.json_normalize(data, record_path =['images'])

images_annotations_labelled_training

,file_name,width,height,id
0,trainval_images/0/0.jpg,500,388,0
1,trainval_images/0/1.jpg,500,375,1
2,trainval_images/0/2.jpg,500,375,2
3,trainval_images/0/3.jpg,500,331,3
4,trainval_images/0/4.jpg,500,387,4
...,...,...,...,...
3954,trainval_images/199/1.jpg,500,375,3954
3955,trainval_images/199/2.jpg,500,333,3955
3956,trainval_images/199/3.jpg,500,375,3956
3957,trainval_images/199/4.jpg,500,375,3957


Concatenating DataFrames 

In [10]:
training_labelled = pd.concat([annotations_labelled_training , images_annotations_labelled_training.drop(['id'], axis = 1)], axis = 1)
training_labelled

,image_id,id,category_id,file_name,width,height
0,0,0,0,trainval_images/0/0.jpg,500,388
1,1,1,0,trainval_images/0/1.jpg,500,375
2,2,2,0,trainval_images/0/2.jpg,500,375
3,3,3,0,trainval_images/0/3.jpg,500,331
4,4,4,0,trainval_images/0/4.jpg,500,387
...,...,...,...,...,...,...
3954,3954,3954,199,trainval_images/199/1.jpg,500,375
3955,3955,3955,199,trainval_images/199/2.jpg,500,333
3956,3956,3956,199,trainval_images/199/3.jpg,500,375
3957,3957,3957,199,trainval_images/199/4.jpg,500,375


## Labelled validation annotations

Showing the **annotation of labelled validation images** from the annotation file [anno_val.json](/content/annotation/annotation/anno_val.json).

In [11]:
import json
import pandas as pd
from pandas import json_normalize

file = ANNOTATION_DIR + 'annotation/anno_val.json'

# load data using Python JSON module
with open(file,'r') as f:
    data = json.loads(f.read())
# Flatten data
annotations_labelled_validation = pd.json_normalize(data, record_path =['annotations'])

annotations_labelled_validation

,image_id,id,category_id
0,0,0,0
1,1,1,0
2,2,2,0
3,3,3,0
4,4,4,0
...,...,...,...
1995,1995,1995,199
1996,1996,1996,199
1997,1997,1997,199
1998,1998,1998,199


Showing the **images annotation of labelled validation images** from the annotation file [anno_val.json](/content/annotation/annotation/anno_val.json).

In [12]:
import json
import pandas as pd
from pandas import json_normalize

file = ANNOTATION_DIR + 'annotation/anno_val.json'

# load data using Python JSON module
with open(file,'r') as f:
    data = json.loads(f.read())
# Flatten data
images_annotations_labelled_validation = pd.json_normalize(data, record_path =['images'])

images_annotations_labelled_validation

,file_name,width,height,id
0,trainval_images/0/30.jpg,500,278,0
1,trainval_images/0/31.jpg,500,333,1
2,trainval_images/0/32.jpg,375,500,2
3,trainval_images/0/33.jpg,500,375,3
4,trainval_images/0/34.jpg,500,375,4
...,...,...,...,...
1995,trainval_images/199/11.jpg,500,375,1995
1996,trainval_images/199/12.jpg,500,333,1996
1997,trainval_images/199/13.jpg,500,333,1997
1998,trainval_images/199/14.jpg,500,333,1998


Concatenating DataFrame

In [13]:
validation_labelled = pd.concat([annotations_labelled_validation , images_annotations_labelled_validation.drop(['id'], axis = 1)], axis = 1)
validation_labelled

,image_id,id,category_id,file_name,width,height
0,0,0,0,trainval_images/0/30.jpg,500,278
1,1,1,0,trainval_images/0/31.jpg,500,333
2,2,2,0,trainval_images/0/32.jpg,375,500
3,3,3,0,trainval_images/0/33.jpg,500,375
4,4,4,0,trainval_images/0/34.jpg,500,375
...,...,...,...,...,...,...
1995,1995,1995,199,trainval_images/199/11.jpg,500,375
1996,1996,1996,199,trainval_images/199/12.jpg,500,333
1997,1997,1997,199,trainval_images/199/13.jpg,500,333
1998,1998,1998,199,trainval_images/199/14.jpg,500,333


## Test annotations

Showing the **images annotation of test images** from the annotation file [anno_test.json](/content/annotation/annotation/anno_test.json).

**NOTE - Since it is the test data, it has no annotations given in the annotations file, for we have to predict those.**

In [14]:
import json
import pandas as pd
from pandas import json_normalize

file = ANNOTATION_DIR + 'annotation/anno_test.json'

# load data using Python JSON module
with open(file,'r') as f:
    data = json.loads(f.read())
# Flatten data
images_annotations_test = pd.json_normalize(data, record_path =['images'])

images_annotations_test

,file_name,width,height,id
0,test/0.jpg,500,375,0
1,test/1.jpg,500,375,1
2,test/2.jpg,500,474,2
3,test/3.jpg,500,375,3
4,test/4.jpg,500,295,4
...,...,...,...,...
7995,test/7995.jpg,500,287,7995
7996,test/7996.jpg,500,333,7996
7997,test/7997.jpg,500,375,7997
7998,test/7998.jpg,500,333,7998


## Dataset Split into training and validation 

Splitting the [trainval_images](/content/trainval_images/trainval_images) dataset(containing both the training and validation images) into training and validation dataset according to the **file_name** column in the **training_labelled** and **validation_labelled** concatenated annotation dataframe.

### Training Split

Creating seperate directory for training dataset named **train**. Copying the training image files from [trainval_images](/content/trainval_images/trainval_images) and pasting to [train](/content/train/train) folder. 

In [15]:
!mkdir train
!mkdir train/train

In [16]:
import os
  
TRAIN_DIR = '/content/train/train/'
  
list = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199]

list_string = [str(x) for x in list]
# list_string
  
for items in list_string:
    train_category_dirs = os.path.join(TRAIN_DIR, items)
    os.mkdir(train_category_dirs)

In [17]:
source_path = '/content/trainval_images/trainval_images/'
destination_path = '/content/train/train/'

In [18]:
import shutil

training = training_labelled['file_name'].str.replace(r'trainval_images/', '')
# training

for i, row in enumerate(training):
  filename = row
  source = os.path.join(source_path, filename) 
  destination = os.path.join(destination_path, filename)
  shutil.copy(source, destination)
  # print(destination)

### Validation Split

Creating seperate directory for validation dataset named **val**. Copying the validation image files from [trainval_images](/content/trainval_images/trainval_images) and pasting to [val](/content/val/val) folder. 

In [19]:
!mkdir val
!mkdir val/val

In [20]:
import os
  
VAL_DIR = '/content/val/val/'
  
list = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199]

list_string = [str(x) for x in list]
# list_string
  
for items in list_string:
    val_category_dirs = os.path.join(VAL_DIR, items)
    os.mkdir(val_category_dirs)

In [21]:
source_path = '/content/trainval_images/trainval_images/'
destination_path = '/content/val/val/'

In [22]:
import shutil

validation = validation_labelled['file_name'].str.replace(r'trainval_images/', '')
# validation

for i, row in enumerate(validation):
  filename = row
  source = os.path.join(source_path, filename) 
  destination = os.path.join(destination_path, filename)
  shutil.copy(source, destination)
  # print(destination)

## Build an input pipeline with tf.data

In [23]:
# Import necessary libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import applications
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

### Training input pipeline

In [24]:
training_labelled['file_name'] = training_labelled['file_name'].str.replace(r'trainval_images/', '')
training_labelled['file_name'] = '/content/train/train/' + training_labelled['file_name'].str[:]
training_dataframe = training_labelled.drop(['image_id', 'id', 'width', 'height'], axis = 1)
training_dataframe

,category_id,file_name
0,0,/content/train/train/0/0.jpg
1,0,/content/train/train/0/1.jpg
2,0,/content/train/train/0/2.jpg
3,0,/content/train/train/0/3.jpg
4,0,/content/train/train/0/4.jpg
...,...,...
3954,199,/content/train/train/199/1.jpg
3955,199,/content/train/train/199/2.jpg
3956,199,/content/train/train/199/3.jpg
3957,199,/content/train/train/199/4.jpg


In [25]:
# Coverting DataFrame to list to pass it to tf.data dataloader

file_name = training_dataframe['file_name'].to_list()
label = training_dataframe['category_id'].to_list()

In [26]:
train_slice = tf.data.Dataset.from_tensor_slices((file_name, label))

In [27]:
for feature_batch, label_batch in train_slice.take(1):
  print("'category': {}".format(label_batch))
  print("'feature': {}".format(feature_batch))

'category': 0
'feature': b'/content/train/train/0/0.jpg'


In [28]:
# Decode the images and resize the images to the mentioned shape

@tf.function
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.io.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize(image, [256, 256])
    return image, label

In [ ]:
# def train_preprocess(image, label):
#     image = tf.image.random_flip_left_right(image)

#     image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
#     image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

#     #Make sure the image is still in [0, 1]
#     image = tf.clip_by_value(image, 0.0, 1.0)

#     return image, label

In [29]:
AUTOTUNE = tf.data.AUTOTUNE

training_dataset = train_slice.shuffle(len(file_name))
training_dataset = training_dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
# training_dataset = training_dataset.map(train_preprocess, num_parallel_calls=AUTOTUNE)
training_dataset = training_dataset.batch(32)
training_dataset = training_dataset.prefetch(AUTOTUNE)

In [30]:
inputs, labels = next(iter(training_dataset))
labels

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([167, 121, 118,  68, 145,  56,  91, 106,  40,  35,  35,  48, 134,
        99,   9,   2,   1, 101, 180,  49,  88,   3, 169,  89,  43,  80,
        22, 154,  72, 111,  46,  49], dtype=int32)>

### Validation input pipeline

In [31]:
validation_labelled['file_name'] = validation_labelled['file_name'].str.replace(r'trainval_images/', '')
validation_labelled['file_name'] = '/content/val/val/' + validation_labelled['file_name'].str[:]
validation_dataframe = validation_labelled.drop(['image_id', 'id', 'width', 'height'], axis = 1)
validation_dataframe

,category_id,file_name
0,0,/content/val/val/0/30.jpg
1,0,/content/val/val/0/31.jpg
2,0,/content/val/val/0/32.jpg
3,0,/content/val/val/0/33.jpg
4,0,/content/val/val/0/34.jpg
...,...,...
1995,199,/content/val/val/199/11.jpg
1996,199,/content/val/val/199/12.jpg
1997,199,/content/val/val/199/13.jpg
1998,199,/content/val/val/199/14.jpg


In [32]:
# Coverting DataFrame to list to pass it to tf.data dataloader

val_file_name = validation_dataframe['file_name'].to_list()
val_label = validation_dataframe['category_id'].to_list()

In [33]:
val_slices = tf.data.Dataset.from_tensor_slices((val_file_name, val_label))

In [34]:
for feature_batch, label_batch in val_slices.take(1):
  print("'category': {}".format(label_batch))
  print("'feature': {}".format(feature_batch))

'category': 0
'feature': b'/content/val/val/0/30.jpg'


In [35]:
AUTOTUNE = tf.data.AUTOTUNE

# validation_dataset = val_slices.shuffle(len(val_file_name))
validation_dataset = val_slices.map(parse_function, num_parallel_calls=AUTOTUNE)
# validation_dataset = validation_dataset.map(train_preprocess, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.batch(32)
validation_dataset = validation_dataset.prefetch(AUTOTUNE)

In [36]:
inputs, labels = next(iter(training_dataset))
labels

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([ 58,  64,   4,  63, 183, 163,  49, 162,   2,  94, 144, 146,  54,
       178,  96,   2,  32,  30,   8, 143,  36, 124,  62,  43, 112, 138,
        54,  11, 108,  84,  75,   3], dtype=int32)>

## Build Model Architecture

In [37]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
  
base_model.trainable = True

model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.GlobalMaxPooling2D(),
                             tf.keras.layers.Dense(200, activation='softmax')
])

model.summary() 

58900480/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 200)               102600    
                                                                 
Total params: 14,817,288
Trainable params: 14,817,288
Non-trainable params: 0
_________________________________________________________________


## Compile and Train the Model

In [38]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
history = model.fit(
        training_dataset,
        epochs = 10,
        validation_data = validation_dataset,
        callbacks=[
          # Stopping our training if val_accuracy doesn't improve after 20 epochs
          tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
    ]
)

Epoch 1/10
124/124 [==============================] - 178s 1s/step - loss: 5.3475 - accuracy: 0.0088 - val_loss: 5.2985 - val_accuracy: 0.0050
Epoch 2/10
124/124 [==============================] - 146s 1s/step - loss: 5.2878 - accuracy: 0.0099 - val_loss: 5.2992 - val_accuracy: 0.0050
Epoch 3/10
124/124 [==============================] - 146s 1s/step - loss: 5.2808 - accuracy: 0.0106 - val_loss: 5.3001 - val_accuracy: 0.0050
Epoch 4/10
124/124 [==============================] - 146s 1s/step - loss: 5.2745 - accuracy: 0.0096 - val_loss: 5.3014 - val_accuracy: 0.0050
Epoch 5/10
124/124 [==============================] - 146s 1s/step - loss: 5.2689 - accuracy: 0.0101 - val_loss: 5.3028 - val_accuracy: 0.0050
Epoch 6/10
124/124 [==============================] - 146s 1s/step - loss: 5.2638 - accuracy: 0.0104 - val_loss: 5.3045 - val_accuracy: 0.0050


## Evaluation

## Testing